# Lab 4 - RAG

## Setup Environment
The following code loads the environment variables, images for the RAG App, and libraries required to run this notebook.


In [ ]:
FILE="GenAI Lab 4"

# ! pip install -qqq git+https://github.com/elastic/notebook-workshop-loader.git@main
from notebookworkshoploader import loader
import os
from dotenv import load_dotenv

if os.path.isfile("../env"):
    load_dotenv("../env", override=True)
    print('Successfully loaded environment variables from local env file')
else:
    loader.load_remote_env(file=FILE, env_url="https://notebook-workshop-api-voldmqr2bq-uc.a.run.app")

In [ ]:
# ! pip install -qqq tiktoken==0.5.2 cohere==4.38 openai==1.3.9
# ! pip install -qqq streamlit==1.30.0 elasticsearch==8.12.0 elastic-apm==6.20.0 inquirer==3.2.1 python-dotenv==1.0.0
# ! pip install -qqq elasticsearch-llm-cache==0.9.5
! echo "github codespaces has pre-installed these libraries"

## <font color=Green>Labs</font>


### <font color=Orange>Lab 4.1 - Gathering Semantic documents from Elasticsearch</font>
This first exercise will allow us to see an example of returing semantically matching documents from Elasticsearch.

It is not too important to understand all the Elasticsearch DSL syntax at this stage.



#### Run the code block below to set up the query function
---



In [ ]:
import os
import openai
from elasticsearch import Elasticsearch
import time
import json
import textwrap


index = os.environ['ELASTIC_INDEX_DOCS_W']

# Create Elasticsearch Connection
es = Elasticsearch(
            cloud_id=os.environ['ELASTIC_CLOUD_ID_W'],
            api_key=(os.environ['ELASTIC_APIKEY_ID_W']),
            request_timeout=30
            )


# Search Function
def es_hybrid_search(question):
    query = {
      "nested": {
        "path": "passages",
        "query": {
          "bool": {
            "must": [
              {
                "match": {
                  "passages.text": question
                }
              }
            ]
          }
        }
      }
    }

    knn = {
      "inner_hits": {
        "_source": False,
        "fields": [
          "passages.text"
        ]
      },
      "field": "passages.embeddings",
      "k": 5,
      "num_candidates": 100,
      "query_vector_builder": {
        "text_embedding": {
          "model_id": "sentence-transformers__all-distilroberta-v1",
          "model_text": question
        }
      }
    }

    rank = {
      "rrf": {}
    }

    fields = [
      "title",
      "text"
    ]

    size = 5

    resp = es.search(index=index,
                  #query=query,
                  knn=knn,
                  fields=fields,
                  size=size,
                  #rank=rank,
                  source=False
                  )

    title_text = []
    for doc in resp['hits']['hits']:
      title_text.append( { 'title' : doc['fields']['title'][0],
        'passage' : doc['inner_hits']['passages']['hits']['hits'][0]['fields']['passages'][0]['text'][0] }
                         )

    return title_text

#### Example Semantic Search With Elastic

In [ ]:
user_question = "Who is Batman?"

In [ ]:
es_augment_docs = es_hybrid_search(user_question)

print('Wikipedia titles returned:\n')
for hit, wiki in enumerate(es_augment_docs):
  print(f"{hit} - {wiki['title'] }" )

### <font color=Orange>Lab 4.2 - Sending Elasticsearch docs with a prompt for a RAG response</font>

#### Run the code below to set up the LLM Connection

In [ ]:
import openai
from openai import OpenAI
import textwrap


# Configure OpenAI client
openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.default_model = os.environ['OPENAI_API_ENGINE']
openai.verify_ssl_certs = False
client = OpenAI(api_key=openai.api_key, base_url=openai.api_base)

if os.environ['ELASTIC_PROXY'] != "True":
    openai.api_type = os.environ['OPENAI_API_TYPE']
    openai.api_version = os.environ['OPENAI_API_VERSION']


# Text wrapper for colab readibility
def wrap_text(text):
    wrapped_text = textwrap.wrap(text, 70)
    return '\n'.join(wrapped_text)


# Function to connect with LLM
def chat_gpt(client, question, passages):

    system_prompt="You are a helpful assistant who answers questions from provided Wikipedia articles."
    user_prompt = f'''Answer the followng question: {question}
                    using only the wikipedia `passages` provided.
                    If the answer is not provided in the `passages` respond ONLY with:
                    "I am unable to answer the user's question from the provided passage" and nothing else.

                  passages: {passages}

                  AI response:
                  '''

    # Prepare the messages for the ChatGPT API
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}]

    response = client.chat.completions.create(model=openai.default_model,
                                              temperature=0.2,
                                              messages=messages,
                                              )
    return response
#    return response.choices[0].message.content

#### Pass the full prompt and wiki passages to LLM

In [ ]:
ai = chat_gpt(client, user_question, es_augment_docs)
print(f"User Question: \n{user_question}\n")
print("AI response:")
print(wrap_text(ai.choices[0].message.content))

### <font color=Orange>Lab 4.3 - Full RAG Application with UI</font>


## Main Script
We've placed the sample code in the streamlit folder of this repository

Take a look at the code [streamlit/app.py](../streamlit/app.py)

## Streamlit
To start the Streamlit app you need to use the ```streamlit run``` command from the folder.  You can do this either from this notebook or the Visual Studio Code terminal provided in Github Codespaces

In [ ]:
! cd ../streamlit; streamlit run app.py 